In [5]:
# -*- coding: utf-8 -*-
"""dna-classification-vcu.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/13BLlKCJc5EiKMHP7KEH5hOXyLEIUdX02

# DNA Multi Class Classification
"""


"""## Prepare Google Drive"""

# Run this cell to mount your Google Drive.

local_path = './'

"""## Prepare fastai"""
from fastai import *
from fastai.text import *

"""## Prepare Dataset"""

local_project_path = local_path + 'dna-10class/'

if not os.path.exists(local_project_path):
  os.makedirs(local_project_path)
  
print('local_project_path:', local_project_path)
"""## Create Language Model"""

class dna_tokenizer(BaseTokenizer):
  def tokenizer(slef, t):
    return list(t)

tokenizer = Tokenizer(tok_func=dna_tokenizer, pre_rules=[], post_rules=[], special_cases=[])
processor = [TokenizeProcessor(tokenizer=tokenizer, include_bos= False, include_eos=False), NumericalizeProcessor(max_vocab=30000)]

# batch size
bs = 64

data_lm = TextLMDataBunch.from_csv(local_project_path, 'combined.csv',
                                   text_cols ='Text', valid_pct= 0.1, tokenizer=tokenizer,
                                   include_bos= False, include_eos=False)
# learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3, pretrained=False).to_fp16()
# learn.load('lm-fine-tuned-10-6')
# learn.save_encoder('lm-fine-tuned-10-6-encoder')

"""## Create Classifier"""
# data_cls = TextClasDataBunch.from_csv(local_project_path, 'combined_train.csv',
#                                    text_cols ='Text', label_cols ='class', valid_pct= 0.1, tokenizer=tokenizer,
#                                    include_bos= False, include_eos=False, vocab = data_lm.vocab)
data_cls = (TextList.from_csv(local_project_path, 'combined.csv', cols='Text', vocab=data_lm.vocab, processor= processor)
                   .split_from_df(col='is_test')
                   .label_from_df(cols='class')
                   .databunch(bs=bs))

print('data_cls validation set size', len(data_cls.valid_ds))
# data_cls.show_batch()

learn_cls = text_classifier_learner(data_cls, AWD_LSTM, drop_mult=0.3, pretrained=False)#.to_fp16()

data_cls validation set size 5314


In [6]:
learn_cls.load('fifth');

In [7]:
dna_string = 'atggcagtggcaggtaaaaatgactttgcagttctcaacaccgggcggaagatgcctctccttgggctgggaacatggaagagtgaacctggcaaggttaaacaggcagtaatctgggccttgcaggctggctaccgccacttcgactgtgctgccatctatggcaacgagttggagatcggagaagctctgcaggagacacttggccctgacaaagccttgaggcgagaggatgtgtttatcacctccaagctgtggaacacacagcatcacccggaggatgtggagcccgctctgctgaagacactgaaggagctgagtctggaatacctggatctatacctcatccactggccctatgccttccaacaaggtgacgctcctttccccaaatcggaggacggcaccctgctgtacgacgacatcgactacaagctgacttgggctgccatggagaagctggtgggaaagggcctggtcagggctatcggcctgtccaacttcaacagcaaacagatagacaacgttctctccgtagccaacatcaaaccgactgtgcttcaggtggaaagccatccgtatctggctcaggtggagttgctgggacactgccgggacagaggcctggtgattacagcgtacagcccactggggtcaccggatcgggtatggaagcatcctgatgagcccgtcctcctggatgaagcagcaatcgacaccctggccaagaagtacaacaagtccccagcacaaatcatccttagatggcagacacagcgaggagtagtgacgatccctaaaagtgtgacagagtctcggatcaaagagaatattcaggtatttgactttacccttgaagcggaagagatgaagtgtataacagcattgaacagaggctggcgctacattgtaccaaccatcacagttgatgggaagcccgtccccagggatgcaggacatccacactaccccttcagtgacccctactga'
res = learn_cls.predict(dna_string)
print(res)


(Category 1.1.1.2, tensor(0), tensor([1.0000e+00, 5.5785e-12, 7.2570e-10, 2.2010e-10, 2.3871e-11, 2.1388e-10,
        7.2346e-11, 1.3818e-08, 1.9142e-07, 8.8164e-09]))


In [9]:
interp = ClassificationInterpretation.from_learner(learn_cls)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

KeyboardInterrupt: 

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)